# Домашнее задание 4. Регрессия
**Дедлайн 23:59 07.12.2018**



# Задача
[Данные](https://www.kaggle.com/sohier/calcofi#bottle.csv) представляют собой различные показания океанической воды, взятые в разных местах. Задача состоит в исследовании того, насколько возможным является восстановление температуры воды по её химическим показателям

**Целевая переменная** - *T_degC* (температура воды в Цельсиях)

# Условие
В рамках данной задачи Вам предлагается попробовать следующее:
1. Предложите метрику качества Вашей модели. Нужны ли нам здесь ассиметричные метрики (аля quantile regression)? Объясните свой выбор.
2. Попробуйте различные модели регрессии для данной задачи. Выберите и затюньте наилучшую.
3. Какие из этих моделей требуют нормализации данных? 
4. Какое качество получилось? Можно ли говорить о какой-либо значимой зависимости температуры воды от её химических показателей?

# Решение

## 0. Загрузка и предобработка данных

In [2]:
import pandas as pd
df = pd.read_csv('../data/bottle.csv.zip')

/home/artem/anaconda3/envs/pytorch_3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (47,73) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
df.isna().sum()/df.shape[0] # доля пропущенных значений

Cst_Cnt                0.000000
Btl_Cnt                0.000000
Sta_ID                 0.000000
Depth_ID               0.000000
Depthm                 0.000000
T_degC                 0.012676
Salnty                 0.054753
O2ml_L                 0.195016
STheta                 0.060922
O2Sat                  0.235400
Oxy_µmol/Kg            0.235407
BtlNum                 0.862791
RecInd                 0.000000
T_prec                 0.012676
T_qual                 0.973259
S_prec                 0.054753
S_qual                 0.913381
P_qual                 0.220969
O_qual                 0.786468
SThtaq                 0.923892
O2Satq                 0.748172
ChlorA                 0.739529
Chlqua                 0.260963
Phaeop                 0.739530
Phaqua                 0.260958
PO4uM                  0.522101
PO4q                   0.477621
SiO3uM                 0.590581
SiO3qu                 0.409310
NO2uM                  0.609677
                         ...   
MeanAs  

Данные очень сырые. Выкинем те объекты, которые содержат пропущенную целевую переменную

In [9]:
df = df[~df['T_degC'].isnull()]

In [22]:
import numpy as np
df_num = df.select_dtypes(include=[np.float]) # оставим только числовые признаки
df_num.head()

,T_degC,Salnty,O2ml_L,STheta,O2Sat,Oxy_µmol/Kg,BtlNum,T_prec,T_qual,S_prec,...,R_NH4,R_CHLA,R_PHAEO,R_SAMP,DIC1,DIC2,TA1,TA2,pH2,pH1
0,10.50,33.440,NaN,25.649,NaN,NaN,NaN,1.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10.46,33.440,NaN,25.656,NaN,NaN,NaN,2.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10.46,33.437,NaN,25.654,NaN,NaN,NaN,2.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10.45,33.420,NaN,25.643,NaN,NaN,NaN,2.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10.45,33.421,NaN,25.643,NaN,NaN,NaN,2.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
X, y = df_num.drop('T_degC',  1), df_num['T_degC']
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)